## Постановка задачи


<b>Цель: </b>

Сопоставление произвольных гео названий с унифицированными именами geonames для внутреннего использования Карьерным центром.

<b>Используемые для решения задачи данные:</b>

В рамках данной задачи, были использованы данные, взятые с сайта <a>http://download.geonames.org/export/dump/</a>, а именно файлы alternateNamesV2, cities15000, countryInfo. Заметим, что эти данные использовались именно для построения решения задачи. Заказчик может использовать в практических целях другие данные. Это необходимо учесть в процессе решения задачи. Кроме того, заказчик предоставил файл 
geo_test.csv для тестирования модели. 

<b>Задачи:</b>


- Создать решение для подбора наиболее подходящих названий с geonames. Например Ереван -> Yerevan. Помимо городов, там могут быть и страны.


- На примере РФ и стран наиболее популярных для релокации - Беларусь, Армения, Казахстан, Кыргызстан, Турция, Сербия. Города с населением от 15000 человек (с возможностью масштабирования на сервере заказчика)


- Возвращаемые поля geonameid, name, region, country, cosine similarity
- формат данных на выходе: список словарей, например [{dict_1}, {dict_2}, …. {dict_n}] где словарь - одна запись с указанными полями



<b>Дополнительные задачи:</b>


- возможность настройки количества выдачи подходящих названий (например в параметрах метода)


- коррекция ошибок и опечаток. Например Моченгорск -> Monchegorsk


- хранение в PostgreSQL данных geonames


- хранение векторизованных промежуточных данных в PostgreSQL


- предусмотреть методы для настройки подключения к БД


- предусмотреть метод для инициализации класса (первичная векторизация geonames)


- предусмотреть методы для добавления векторов новых гео названий


## Описание проекта и пайплайн решения

### Описание исспользуемых данных

- cities15000 : таблица, содержащая унифицированные названия городов и их уникальный идентефекатор, а также код страны (например, RU для России) и региона. В рамках задачи используются только те города, которые относятся к перечисленным выше странам.
- alternateNamesV2 : таблица, содержащая некоторое количество альтернативных названий для каждого города (названия на разных языках, исторические названия и т.п.), а так же их уникальный идентефекатор. В рамках задачи используются только те города, которые относятся к перечисленным выше странам.
- countryInfo : таблица, содержащая названия стран и их коды.
- geo_test.csv : таблица, содержащая "запрос" и истинное название города. Нужна для тестирования решения.
- admin1CodesASCII : таблица, необходимая для извлечения названий регионов.

### Описание решения задачи

Суть задачи сводится к тому, чтобы сопоставить введённое пользователем слово со словами из таблицы с унифицировннаыми названиями (в нашем случае, cities15000), и выбрать топ-k наиболее близких к нему, где k определяется заказчиком (например, топ-5 или любое другое количество). 

Для этого было решено предварительно векторизовать слова из спика унифицированных названий, а также введённое пользователем слово и затем искать косинусное сходство между векторами. Далее мы узнаём ближайшие топ-k geoname_id и по ним ищем унифицированные названия городов и другую необходимую информацию. Можно было бы векторизовать все слова из таблицы alternateNamesV2, но даже с учётом ограниченной выборки, поиск по векторам занимал бы большое количество времени (порядка 40-60 секунд для одного предсказания), что слишком много для практического применения.

Для создания веторов было решено использовать предобученные нейронные сети Sentence Transformers. Эти нейронные сети уже обучены на большом количестве текстов, и даже без дополнительного обучения, некоторые из них способны решить задачи. Они находятся на сайте <a>https://huggingface.co/sentence-transformers</a>. 

В рамках решения задачи была выбрана нейросеть <a>https://huggingface.co/sentence-transformers/distiluse-base-multilingual-cased-v2</a>. Она показывает неплохие результаты, и кроме того, достаточно "лёгкая" для того, чтобы её можно было дообучить с имеющимися мощностями. Модель <a>https://huggingface.co/sentence-transformers/LaBSE</a> показывала лучшие результаты "из коробки", то есть без дообучения, однако для того, чтобы её дообучать, необходимы недоступные вычислительные мощносити. В результате, выбранная модель с учётом дообучения победила.

Для улучшения показателй было решено дообучить её на выборке из таблицы alternateNamesV2, сопоставив альтернативные и истинные названия. Кроме того, для увеличения эффективности обучения было решено предобработать данные, приведя их к единому виду (нечто вроде лемматизации) и расширить обучающую выборку, добавив в неё также альтернативные названия со случайными опечатками, сгенерированными скриптом. 

<b>Стадии решения задачи:</b>

- Загрузка необходимых данных из датабазы : соединение с postgreSQL базой данных, создания запросов, которые выгрузят необходимую выборку по странам. Обзор данных.

- Подготовка данных для обучения : лемматизация городов из alternateNamesV2; сопоставление их с унифицированными названиеми по geonameid;объединение с названиями стран. Данные по странам были также расширены - туда вручную были внесены альтернативные названия (в основном, перевод названий стран на русский язык) и им искуственно присвоен уникальный geonameid; расширение обучающей выборки методом добавления случайных опечаток.

- Обучение модели и сохранение обученной модели.

- Векторизация городов и стран с помощью обученной модели.

- Тестирование получившейся модели.

- Итоговая функция, возвращающая требуемый список словарей.

### Доролнительные задачи проекта

Помимо решения основной задачи, мы предусмотрели дополнительные пожелания и удобство заказчика. Под этим подразумевается две вещи:

<b>Сохранение данных, требующих вычислений</b>, таких как векторезованные слова в отдельную схему базы данных. Таким образом, заказчику не придётся каждый раз векторизовать слова при запуске проекта. Достаточно сделать это один раз и результаты вычислений будут записаны в базу данных, повторное вычисление векторов потребуется произвести только при изменении данных или модели. То же касается и других вещей: обученной модели и предподготовленных данных. Для этого было решено создать дополнительную схему с одной строчкой, где будет указано, какие расчёты уже были сделаны. Более того, эта схема обновляется автоматически (при желании заказчика).

<b>Отсутствие необходимости вмешиваться в код программы при изменении данных или модели</b>. Мы учли тот факт, что при реальном применении решения задачи, заказчик может использовать другие данные - например, он может захотеть расширить датасет, или банально могут отличаться названия таблиц и полей. Кроме того, изменение данных может потребовать повторного переобучения модели, создания векторов и так далее, и даже выбора другой модели или параметров обучения. 

Программа была спректирована таким образом, чтобы пользователю было достаточно изменить значения некоторых константных переменных, чтобы подстроить программу под свои нужды. 

В файле <b>setup.py</b> находятся все переменные, связанные с базой данных - параметры, необходимые для подключения к БД (пароль, хост и т.п.), а также названия таблиц и использованных полей. Фактически, пользователь может загрузить любые данные с любыми названиями, лишь бы их структура соотвествовала тем, что были предоставлены при решении задачи (точнее, необходимые поля).

Переменные, связанные с параметрами обучения модели, а также выбором самой модели, вместе с подробным описанием, находятся сразу после секции с подключением модуля и подключением к БД. 

<i>Таким образом, для использования данного решения, заказчику достаточно лишь поменять значения некоторых констант. </i>

## Загрузка модулей, подключение к БД и инициализация пользовательских параметров

### Подгрузка необходимых модулей

In [145]:
# Нейросеть, используемая для решения задачи
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from sentence_transformers.util import semantic_search
from torch.utils.data import DataLoader
 
# Модуль, необходимый для работы с np.Array
import numpy as np

# Модуль для работы с выгруженными данными в Питоне
import pandas as pd

# Модули для лемматизации (предобработки) слов
import random
# Выбираем сид для того, чтобы при повторном запуске все генерируемые псевдослучайные числа не менялись
random.seed(42)
import unidecode

# Модули для подключения к базе данных
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import psycopg2

# "Декоративный" модуль для отображения полосы прогресса при вычислениях
from tqdm.notebook import tqdm

# Утилита для перезагрузки пользовательских модулей в случае их изменения
import importlib  

# Модуль, содержащий константные переменные, связанные с базой данных, такие как параметры для подключения к БД, название таблиц и полей.
import setup
importlib.reload(setup)
from setup import *


### Подключение к базе данных PostgreSQL

Создадим подключение к базе данных для того, чтобы далее  мы могли обращаться к ней и считывать данные или записывать новые. Все параметры подключения находятся в файле <b>setup.py</b>.

In [146]:
DATABASE = {
    'drivername': DRIVERNAME,
    'username': USERNAME, 
    'password': PASSWORD, 
    'host': HOST,
    'port': PORT,
    'database': DATABASENAME,
    'query': {}
}  

engine = create_engine(URL(**DATABASE))

conn = psycopg2.connect(
    host=HOST,
    database=DATABASENAME,
    user=USERNAME,
    password=PASSWORD
)

### Инициализация пользовательских параметров

Ниже перечислены константы, которые являются настраиваемыми параметров. То есть, для того, чтобы изменить параметры работы программы, достаточно изменить значения констант. Ниже будут приведены все константы и их подробное описание, а также описание того, в каком случае их стоит менять.
Следующая за ними секция также содержит константы, связанные с необходимостью повторных вычислений.

In [147]:
# ОБЩЕЕ #

# Константа содержащая страны, города которых учитываются. По условию задачи, это 
# Россия, Беларусь, Армения, Казахстан, Кыргызстан, Турция, Сербия. Если вы хотите добавить страну, просто добавьте её буквенный код.
# Страну надо добавлять в том же формате, в котором изначальное значение переменной, то есть добаветь к строке в одиночных кавычках.
# Так будет выглядеть переменная, если вы захотите добавить, например, США:
# COUNTRIES =" 'BY', 'AM', 'KZ', 'KG', 'TR', 'RS', 'RU', 'US' "
COUNTRIES ="'BY', 'AM', 'KZ', 'KG', 'TR', 'RS', 'RU'"

# ПАРАМЕТРЫ ФОРМИРОВАНИЯ ОБУЧАЮЩЕЙ ВЫБОРКИ #

# Использование дополнительных альтернативных названий для стран. Данные были сформированы вручную (в основном, посредством перевода).
# Если не хотите их использовать, поставьте значение False. 
MORE_ALTERNATE_COUNTRIES = True

# Если вы хотите использовать дополнительные данные из csv файла:
ALTERNATE_COUNTRIES_CSV = 'data/more_alternate_names_countries.csv'

# Если вы загрузили дополнительные данные по странам в базу, то поставьте этой переменной значение True. Название таблицы в setup.py
ALTERNATE_COUNTRIES_FROM_DATABASE = True

# Расширение обучающей выборки посредством добавления опечаток. От значения переменной зависит то, сколько раз эта процедура будет 
# произведена. Т.е. во время каждой итерации проходим по базовому (без опечаток) обучающему датасету, генерируем опечатки и добавляем
# в итоговый обучающий датасет. Увеличение этого параметра улучшает качество обучения, но и увеличивает объём обучающей выборки в 
# практически (так как опечатки могут изредка повторяться и мы потом убираем дубликаты) в (1 + количество итераций) раз! 
# Это значительно увеличит время обучения. Экспериментальным путём было принято решение остановиться на трёх итерациях, Но вы можете
# изменить этот параметр.
WARPS = 2

# ПАРАМЕТРЫ МОДЕЛИ И ЕЁ ОБУЧЕНИЯ #

# Путь к модели, которую собираетесь обучать. Если вы не собираетесь дообучать вашу модель, эта переменная не имеет значения. 
# Вы можете скачать модели с сайта https://huggingface.co/sentence-transformers, если хотите обучить не ту модель, которая 
# предложена в решении. 
# 
# Чтобы сделать это надо либо склонировать репозиторий в выбранную вами папку и прописать полный путь, как это сделано в проекте.
# либо просто прописать, например
# BASE_MODEL =  'sentence-transformers/LaBSE'. 
# Во втором случае модель установится в папку с Питоном по умолчанию (это может занять много времени, если модель большая) 
# Вы можете указать путь и на уже дообученную модель, чтобы дообучить её дальше.
BASE_MODEL =  './models/distiluse-base-multilingual-cased-v2'

# Путь к модели, которая будет использоваться непосредственно в решении задачи. Также тот путь, куда будет сохранена обученная модель.
# Если вы не хотите дообучать модель, то только ЭТА переменная имеет значения, предыдущая не важна.
# Значение переменной может совпадать с предыдущей.
FINAL_MODEL = './models/distiluse-base-multilingual-cased-v2-4epochs-CITIESFINDER'

# Количество эпох, которые обучается модель. До определённого момента улучшает показатели, но затем может переобучиться, точно предсказать 
# оптимальное число эпох невозможно, это зависит от модели и данных. Одна эпоха может занять много времени!
EPOCHS = 4

# Размер "батча". Чем больше, тем быстрее обучается модель, но при том она требует больше оперативной памяти. 
BATCH_SIZE = 32

# ТЕСТИРОВАНИЕ МОДЕЛИ #

# Путь к тестовому файлу. Тестовый файл содержит поля 'query' с тестовым запросом и 'name' с истинным названием города. 
# Названия должны быть именно такими. Разделитель - ';'.
TEST_CSV = 'data/geo_test.csv'

# Функция тестирования возвращает две метрики: accuracy - процент попадания нужного города в первую строчку вывода модели и
# accuracy*k - процент попаданий нужного города в список из k строк. Этот параметр отвечает за величину k

TEST_K = 5

### Параметры записи результатов вычислений в датабазу и повторных вычислений

Некоторые шаги выполнения программы требуют вычислений и времени, а именно: обучение модели и преобразование слов в векторы. Было решено записать результаты вычислений в отдельные таблицы базы данных (модель сохраняется в папке проекта). Для того, чтобы программа могла понять, нужно ли делать вычисления или они уже были выполнены, было решено добавить вспомогательную таблицу данных в БД, содержащую одну строку, в которой указывается были ли сделаны те или иные шаги. Затем программа считывает эту таблицу и смотрит на то, какие вычисления необходимо выполнить. 

Таблица будет содержать нули в тех полях, где вычисления ещё не сделаны. Таким образом, если в таблице ноль - то программа делает вычисления, если один - то нет. По желанию пользователя таблица может автоматически обновляться после вычислений. 

Если после произведённых вычислений требуется сделать их повторно (например, обучить новую модель), то у пользователя есть два пути:
1. Обновить таблицу прямо в коде программы, вручную выставив необходимые опции и присвоив соответствующему параметру значение True (описание опций и параметра ниже)
2. Обновить таблицу средствами управления базой данных. 

Второй вариант необходим так как позволит автоматизировать обновление таблицы, например, когда те или иные данные обновляются. Скажем, инженер данных со стороны заказчика мог бы написать скрипт, который ставит всем параметрам в таблице значения равные нулю когда дополняется таблица, содержащая альтернативные названия городов на определённое количество данных. Тогда при запуске скрипта модель заново обучится и векторы будут завово созданы, это не придётся каждый раз контролировать вручную. Однако, обновить таблицу можно и вручную.


In [148]:
# Обучение модели. Нужно если вы хотите обучить новую модель или добучить старую. Если значение равно 1, значит модель уже обучена и не будет
# повторно обучаться. Потому если вы хотите обучить модель заново - значение должно быть равно нулю, а не наоборот, как могло показаться!

FIT_MODEL = 0

# Векторизация слов из поисковой выборки. Повторная векторизация необходима если вы заново обучили модель или если вы изменили выборку
# по которой модель ищет истинные названия городов (в нашем случае - cities1500)
VECTORIZATION = 0

# Обновление таблицы с опциями. Если значение равно True, то программа при каждом запуске будет переписывать таблицу с опциями в соответствии 
# со значениями констант, используемых выше. Если False, программа не будет этого делать, соответственно тогда значения констант не важны. 
REFRESH_OPTIONS = True

# После завершения того или иного этапа обновления, программа запомнит это и автоматически добавит в таблицу с опциями значение 1 в 
# соответствующем поле. Если вы не хотите, чтобы она это делала, поставьте константе значение False. 
AUTO_UPDATE_OPTIONS = True

Теперь напишем код, который создаст и считает таблицу с опциями.

In [149]:
# Создадим датафрейм с соответствующими названиями полей и строчкой, заполненной нулями. Он нужен на тот случай, если таблица ещё 
# не была создана вовсе или вы хотите обновить таблицу в программе. 
update_database = pd.DataFrame( columns = ['fit_model', 'vectorization'])

if REFRESH_OPTIONS:
# Если мы хотим менять опции вручную, то присваиваем соответствующим столбцам датафрейма соответствующие значения.
    update_database.loc[0] = [FIT_MODEL, VECTORIZATION]
else: 
# Если опции менять не нужно, присваиваем все нули. Это нужно исключительно на тот случай, если таблица вовсе не была создана.
    update_database.loc[0] = [0, 0]

if REFRESH_OPTIONS:
# Если хотим обновить опции вручную, просто переписываем таблицу с опциями, ставя значение if_exists='replace'.
     update_database.to_sql(OPTIONS, engine, if_exists='replace', index=False)
     print('Таблица с опциями обновлена')
# Иначе проверяем, создана ли наша таблица. Для этого попробуем обновить её с параметром if_exists='fail'. Если таблица существует,
# это вызовет исключение, но мы используем try-except, так что всё в порядке.
else:
    try:
         update_database.to_sql('model_options_table', engine, if_exists='fail', index=False)
         print('Таблица с опциями создана')
    except:
        print('Таблица с опциями уже существует. Чтобы обновить - присвойте константе REFRESH_OPTIONS значение True')

# Считаем таблицу с опциями

query = f'SELECT * FROM "{OPTIONS}"'
options = pd.read_sql_query(query, con=engine)

# Запишем считанные опции в отдельные переменные, так как их немного и объединять их в список или словарь не имеет особого смысла
FIT_MODEL_OPTION = options.iloc[0]['fit_model']
VECTORIZATION_OPTION = options.iloc[0]['vectorization']

Таблица с опциями обновлена


Мы инициализировали пользовательские переменные и опции. Благодаря этому, пользователь сможет настроить программу под себя не вмешиваясь в основной код. Это необходимо также потому, что скорее всего, заказчик будет использовать более полные данные чем те, что он дал для проектирования решения задачи, соответственно, ему придётся обновить вектора и, возможно, заново обучить модель.

## Загрузка и обзор данных

Для решения задачи нам необходимы следующие данные:

- cities15000 : таблица, содержащая унифицированные названия городов. Основная таблица для поиска данных.
- alternateNamesV2 : таблица, содержащая некоторое количество альтернативных названий для каждого города (названия на разных языках, исторические названия и т.п.).
- countryInfo : таблица, содержащая названия стран и их коды.
- admin1CodesASCII : таблица, необходимая для извлечения названий регионов.
- geo_test.csv : таблица, содержащая "запрос" и истинное название города. Нужна для тестирования решения.

Сначала загрузим части датасетов и посмотрим на них. Мы будем делать все запросы здесь и далее с использованием констант. То есть, переменная с текстом запроса будет содержать не строку вида <code>'SELECT * FROM "cities15000" LIMIT 3'</code>, а она будет генерироваться выражением вида <code>'SELECT * FROM "' + CITIES + '" LIMIT 3'</code>, чтобы пользователь мог работать с программой если у него другие названия таблиц. Например, если он использует cities500 вместо cities15000, ему достаточно просто в файле <b>setup.py</b> поменять значение константы  CITIES на соотвествующее.

#### Cities15000

In [150]:
query = f'SELECT * FROM "{CITIES}" LIMIT 3'
pd.read_sql_query(query, con=engine)

,geoname_id,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date
0,3040051,les Escaldes,les Escaldes,"Ehskal'des-Ehndzhordani,Escaldes,Escaldes-Engo...",42.50729,1.53414,P,PPLA,AD,None,08,None,None,None,15853,None,1033,Europe/Andorra,2008-10-15
1,3041563,Andorra la Vella,Andorra la Vella,"ALV,Ando-la-Vyey,Andora,Andora la Vela,Andora ...",42.50779,1.52109,P,PPLC,AD,None,07,None,None,None,20430,None,1037,Europe/Andorra,2020-03-03
2,290594,Umm Al Quwain City,Umm Al Quwain City,"Oumm al Qaiwain,Oumm al Qaïwaïn,Um al Kawain,U...",25.56473,55.55517,P,PPLA,AE,None,07,None,None,None,62747,None,2,Asia/Dubai,2019-10-24


Нас интересуют поля geoname_id, name, asciiname, admin1_code и country_code	для создания выборки по интересующим нас странам. Заметим, что здесь присутствует столбец alternatenames, который содержит альтернативные имена, записанные в виде строчки. Но у нас имеется таблица с альтернативными именами, и она удобней для обучения данных.

#### alternateNamesV2

In [151]:
query = f'SELECT * FROM "{ALTERNATE_NAMES}" LIMIT 3'
pd.read_sql_query(query, con=engine)

,alternatenameid,geoname_id,isolanguage,alternate_name,ispreferredname,isshortName,iscolloquial,ishistoric,from,to
0,1284819,2994701,None,Roc Mélé,None,None,None,None,None,None
1,1284820,2994701,None,Roc Meler,None,None,None,None,None,None
2,4285256,3007683,None,Pic des Langounelles,None,None,None,None,None,None


На самом деле, эта таблица огромная, но нас интересует ограниченная выборка из стран. В ней нас интересуют исключительно столбцы geoname_id и alternate_name. Таблица нужна только для обучения модели, иначе её нет смысл загружать.

#### countryInfo

In [152]:
query = f'SELECT * FROM "{ COUNTRY_INFO}"'
df = pd.read_sql_query(query, con=engine)
df.head(3)

,iso,iso3,isonumeric,fips,country,capital,area,population,continent,tld,currencycode,currencyname,phone,postal_code_format,postal_code _regex,languages,geoname_id,neighbours,equivalentfipscode
0,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU,.ad,EUR,Euro,376,AD,None,None,NaN,None,None
1,AE,ARE,784,AE,United Arab Emirates,Abu Dhabi,82880.0,9630959,AS,.ae,AED,Dirham,971,None,None,"ar-AE,fa,en,hi,ur",290557.0,"SA,OM",None
2,AF,AFG,4,AF,Afghanistan,Kabul,647500.0,37172386,AS,.af,AFN,Afghani,93,None,None,"fa-AF,ps,uz-AF,tk",1149361.0,"TM,CN,IR,TJ,PK,UZ",None


Нас интересуют сами страны (столбец country), а также geoname_id. Эта таблица нам нужна, так как помимо городов, пользователи могут в качестве локации указывать свою страну. Однако, мы сразу видим пропуск в geoname_id. Также в таблице содержится столбец iso, откуда можно было взять коды стран для создания выборки, однако удобней было посмотреть эти коды в интернете, чем искать в таблице.

In [153]:
# Посмотрим на общую длину таблицы со странами и количество пропусков в geoname_id.
len(df[GEONAME_ID_COLUMN]), df[GEONAME_ID_COLUMN].isna().sum()

(252, 158)

Мы видим, что у большинства стран пропуски в колонке geoname_id, что нас не устраивает. Было решено искуственно присвоить им идентефикаторы, что сделаем ниже.

#### admin1CodesASCII

In [154]:
query = f'SELECT * FROM "{ADMIN_CODES}" LIMIT 3'
pd.read_sql_query(query, con=engine)


,code,name,name_ascii,geoname_id
0,AD.06,Sant Julià de Loria,Sant Julia de Loria,3039162
1,AD.05,Ordino,Ordino,3039676
2,AD.04,La Massana,La Massana,3040131


Нас интересует столбец code и name, чтобы извлечь название региона конкретного города, которое нам также нужно по условию задачи.

### Загрузка данных для дальнейшего использования

Мы сделали краткий обзор данных и теперь нам нужно выгрузить конкретно необходимые нам данные, а также подготовить их для того, чтобы использовать нашу модель.

Таблицу <b>Загрузим admin1CodesASCII</b> пока загружать нет смысла, так как она понадобится исключительно в самом конце для финальной функции.

#### Загрузка таблицы со странами
Загрузим для начала страны. Мы видели, что в столюце geoname_id очень много пропусков. Потому мы заполним этот столбец отрицательными значениями, соотвествующие индексу записи в таблице если там пропуск и просто отрицательными значениями, если geoname_id определён.

Это нужно для нашей функции поиска ближайших названий локации, чтобы понять относится ли локация к городам или стране.

In [155]:
# Считываем таблицу со странами. Мы используем константы вместо строк в названиях таблицы и столбцов, чтобы пользователь мог использовать
# другие имена и ему не пришлось бы переписывать код.

# Сама функция нам ещё пригодится
def make_countries(save_country_code = False):
    if save_country_code:
        query = f'SELECT"{COUNTRY_COLUMN}", "{GEONAME_ID_COLUMN}", "{ISO_COLUMN}" FROM "{COUNTRY_INFO}"'
    else:
        query = f'SELECT"{COUNTRY_COLUMN}", "{GEONAME_ID_COLUMN}" FROM "{COUNTRY_INFO}"'
    df_countries = pd.read_sql_query(query, con=engine)
    # Нам надо заполнить пропуски и заменить существующие geo_id на отрицательные значения для удобства написании финальной функции.
    # Напишем функцию, которую потом используем в apply
    def set_geoname_id(country_index, country_id):
    # Определяем пропуски
        if country_id != country_id:
            # Если пропуск, то присваиваем значение равное минус индексу записи в таблице. 
            return -country_index
        else:
            # Если нет пропуска, то просто возвращаем имеющееся значение, умноженное на минус единицу
            return -country_id
    
# Теперь надо применить нашу функцию. Для этого надо во-первых извлечь индексы, во-вторых использовать лямбда функцию, чтобы наша функция
# могла получить значения двух разных столбцов в качестве аргументов.    
    df_countries[GEONAME_ID_COLUMN] = df_countries.reset_index(inplace= False )[['index', GEONAME_ID_COLUMN]].\
    apply(lambda row: set_geoname_id(row['index'], row[GEONAME_ID_COLUMN]), axis = 1)

# Значения geonames_id были float, так как изначально там присутствовали пропуски, переведём в int.
    df_countries[GEONAME_ID_COLUMN] = df_countries[GEONAME_ID_COLUMN].astype(int)
# Посмотрим на результат
    return df_countries

df_countries = make_countries()
display(df_countries.isna().sum())
df_countries.head(5)


country       0
geoname_id    0
dtype: int64

,country,geoname_id
0,Andorra,0
1,United Arab Emirates,-290557
2,Afghanistan,-1149361
3,Antigua and Barbuda,-3576396
4,Anguilla,-3573511


Мы успешно загрузили таблицу со странами и справились с пропусками. 

#### Загрузка таблицы cities15000

Теперь нам нужно загрузить данные из таблицы cities15000, но только те, что соответствуют нужным по условиям задачи странам. Это Россия, Беларусь, Армения, Казахстан, Кыргызстан, Турция и Сербия. Они уже записаны в константе COUNTRIES.

In [156]:
# Формируем запрос, который выберет только те города, в которых значение country_code равно коду одной из стран, которые мы используем.
query = 'SELECT * FROM "' + CITIES  + '" WHERE"' +COUNTRY_CODE_COLUMN + '"'+ ' IN (' + COUNTRIES + ')'
df_cities = pd.read_sql_query(query, con=engine)

# Если честно, я изначально во всём коде использовал именно колонку с именем, только потом сообразил, 
# что целесообразней использовать asciiname. Чтобы не перелопачивать весь код, я просто присвоил колонке с именами нужное мне значение
df_cities[NAME_COLUMN] = df_cities[ASCII_NAME_COLUMN]

# Посмотрим, есть ли пропуски в интересующих нас столбцах. Столбец country_code проверять не нужно, так как даже если там были пропуски,
# они бы не попали в наш запрос.
df_cities[[NAME_COLUMN, GEONAME_ID_COLUMN, ADMIN_CODES_COLUMN]].isna().sum(), df_cities.shape

(name           0
 geoname_id     0
 admin1_code    0
 dtype: int64,
 (1711, 19))

Пропусков нет. Создадим новую переменную df_cities_emb, которую позже будем использовать для поиска косинусных расстояний. В переменной будут храниться те же данные, что в df_cities, но поля только с именем и geoname_id. Позже в неё добавим страны. После эту переменную превратим в векторы и именно от неё зависит, среди каких названий модель будет искать сходства.

In [157]:
df_cities_emb = df_cities[[NAME_COLUMN,GEONAME_ID_COLUMN]].copy()
df_cities_emb.head(1)

,name,geoname_id
0,Kapan,174875


 #### Загрузка альтернативных названий городов

Нам осталось загрузить альтернативные имена городов. Нам нужны альтернативные названия только тех городов, которые находятся в списке интересующих нас стран. Но в самой таблице с альтернативными названиями нет столбца с кодом страны, есть только столбец с geoname_id, а потому нам нужно извлечь все geoname_id из списка наших городов и использовать их как фильтр.

In [158]:

query = (
    f'SELECT "{ALTERNATE_NAME_COLUMN }","{GEONAME_ID_COLUMN}" FROM "{ALTERNATE_NAMES}" '
    f'WHERE "{GEONAME_ID_COLUMN}" IN ( '
    f'SELECT "{GEONAME_ID_COLUMN}" FROM "{CITIES}" '
    f'WHERE "{COUNTRY_CODE_COLUMN}" IN ({COUNTRIES}) ) '
)
df_alternate_names = pd.read_sql_query(query, con=engine)
display(df_alternate_names.head(10))



,alternate_name,geoname_id
0,Qafan,174875
1,Kapan,174875
2,Kapan,174875
3,Kapan,174875
4,کاپان,174875
5,Կապան,174875
6,Капан,174875
7,https://en.wikipedia.org/wiki/Kapan,174875
8,https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D0%...,174875
9,Капан,174875


Альтернативные названия загрузились. Мы видим, что здесь присутствуют названия на разных языках, в том числе с нестандартной раскладкой. Мы напишем функцию, которая стандартизирует слова, переводя раскладку на один язык.

 Кроме того, здесь есть альтернативные названия вроде "https://en.wikipedia.org/wiki/Kapan", что явно лишнее. Это необходимо будет удалить.

### Добавление стран к обучающей выборки и выборки для создания векторов

Мы хотим, чтобы наша модель могла искать не только названия городов, но и стран. Для этого нам надо соединить страны с обучающей выборкой (если создаём) и df_cities_emb, чтобы модель могла искать страны. 

Также нами была вручную создана дополниьтельная таблица с альтернативными названиями стран. В основном, туда добавлены переведённые с помощью Яндекс Переводчика на русский язык названия стран, но также были добавлены и другие названия для некоторых стран, например, для России были добавлены такие названия, как РФ и Российская Федерация.

Альтернативные названия будут использоваться только в обучающей выборке, в выборке для поиска похожих названий будут только унифицированные названия стран. Структура альтернативных названий такова : она имеет только две колонки - country и geoname_id. В колонке country названия стран (оригинальные и альтернативные), в колонке geoname_id - полученные ранее geoname_id (они могут повторяться). Если вы решили использовать эти данные, они добавятся в данные для обучения вместо обычных стран.

In [159]:
# Проверяем, хотите ли вы использовать дополнительные альтернативные названия стран для обучения
if MORE_ALTERNATE_COUNTRIES:
# Проверяем откуда вы предпочитаете взять альтернативные названия стран - из Базы Данных или csv-файла
    if ALTERNATE_COUNTRIES_FROM_DATABASE:
        # Формируем запрос, если из БД
        query = f'SELECT * FROM "{ALTERNATE_COUNTRIES}"'
        df_more_alternate_countries = pd.read_sql_query(query, con=engine)
    else:
        # Считываем из csv-файла если хотите брать из него
        df_more_alternate_countries = pd.read_csv(ALTERNATE_COUNTRIES_CSV, sep = ',')

# Переименовываем колонку country в alternate_name, чтобы затем объединить с таблицей с альтернативными именами.
    df_more_alternate_countries = df_more_alternate_countries.rename(columns = {COUNTRY_COLUMN:ALTERNATE_NAME_COLUMN})
# Объединяем с таблицей с альтернативными именами и получаем новую таблицу
    df_alter = pd.concat([df_more_alternate_countries,df_alternate_names])
 
else:
# Если мы не хотим использовать дополнительные названия стран, то 
# объелиняем выборку с обычными названиями стран
    df_alter = pd.concat([df_countries.rename(columns = {COUNTRY_COLUMN:ALTERNATE_NAME_COLUMN}) ,df_alternate_names])
    

# В любом случае, объединяем df_cities_emb с обычными названиями странам  
df_cities_emb = pd.concat([df_countries.rename(columns = {COUNTRY_COLUMN:NAME_COLUMN}),df_cities_emb])

# Старая переменная нам больше не понадобится, потому удалим её 
del df_alternate_names


Мы успешно объединили обучющую и выборку для векторизации с названиями стран.

## Подготовка обучающих данных

Для обучения мы сформируем выборку, состоящую из двух столбцов - истинные имена и альтернативные имена. Сами модели <b>Sentence-Transformers</b> не ищут похожие слова, они преобразуют слова в векторы. Затем уже с помощью другой функции мы ищем косинусное сходство между векторами и находим самые похожие "по смыслу" слова (или предложения, но в нашем случае именно слова). 

Модель получает на вход пару слов и метку того, на сколько они должны быть похожи. Но поскольку мы обучаем на парах "истинное имя - альтернативное имя", метка всегда будет 1 (то есть слова одинаковы по смыслу). У нас получается выборка где все данные имеют одинаковую метку, но модель <b>Sentence-Transformers</b> позволяет обучаться на таких данных, то есть негативные примеры и не нужны.

Но перед тем как сформируем обучающие данные, напишем две функции для обработки слов, которые увеличат эффективность.

#### Унификация слов

Сначала напишем функцию для лемматизации, или, вернее будет сказать, унификации слов, то есть приведём все слова к одному виду - в нижнем регистре, написаны латиницей и без дополнительных символов и пробелов.

In [160]:
# Функция принимает "сырое" слово и возвращает обработанное.
def prepare_word(word):

# Определим символы, которые мы хотим удалять из слов.
    bad_symbols = '''
    .,-'!`_ 

    '''
# С помощью модуля unidecode переводим любой язык в латинкский алфавит
    clear_word = unidecode.unidecode(word)

# Удаляем все лишние символы
    clear_word = ''.join([i for i in clear_word if i not in bad_symbols])

# Приводим слово к нижнему регистру
    clear_word = clear_word.lower()
    return clear_word


# Проверим работу функции. Список взят из альтернативных имён, он специально подобран так, 
# чтобы тут встречались слова на нестандартных языках
[print (prepare_word(i))for i in ['Кариба','كاريبا ','کاریبا، زمبابوے','卡里巴','카리바']]
;

kariba
kryb
khrybzmbbwy
qialiba
kariba


''

Как видим, функция справилась. Надо учесть что и само звучание слова "Кариба" на других языках может в корне отличаться от привычного нам. Да, получившиеся слова отличаются от kariba, но видно, что это не случайные наборы символов. Таким образом, функция справляется с разными языками.

#### Аугментация данных

Теперь создадим функцию для аугментации данных - то есть синтетического создания новых данных на основе уже имеющихся. В данном случае, мы будем добавлять в слова случайные опечатки. Это как раз соотвествует одной из задач - справляться с опечатками.

In [161]:

# Объявляем функцию

def warp(
        #Слово, которое мы будем менять 
        word, 
        # Определяем минимальную длину слова, в которое будем добавлять опечатки, чтобы не добавлять опечатки в такие слова
        # как "Спб" или "РФ". 
        min_word_length = 4,
        # Количество опечаток, которые мы добавляем к слову
        warps = 1,
        # Тип опечатки которую мы хотим добавить. Значения могут быть следующими:
        # 'replace' - заменить случайный символ из слова на случайный символ
        # 'add' - добавить случайный символ в случайное место слова
        # 'remove' - удалить случайный символ
        # 'shuffle' - поменять местами два соседних символа
        # 'random' - тип опечатки выбирается случайным образом из всех вышеперечисленных методов (это мы и будем использовать)
        char_warp = 'random'):
    
    # Так как мы имеем дело с английским алфавитом, применям унификацию к слову
    word = prepare_word(word)
    # Проверяем длину слова. Если меньше минимально-допустимой, то возвращаем изначальное слово (то есть ничего не делаем с ним)
    if len(word) < min_word_length:
        return word
    
    # Добавляем опечатки
    while warps > 0:
        # Определяем переменную, которая принимает значение случайной буквы из английского алфавита
        random_char = random.choice(list('abcdefghijklmnopqrstuvwxyz'))
        # Если char_warp = random то выберем случайный метод опечатки, иначе тот, который мы указали. Запишем его в новую переменную
        # чтобы при повторной итерации метод снова выбирался случайно (если мы установили соответствующее значение)
        if char_warp == 'random':
            char_warp_select = random.choice(['replace', 'add', 'remove', 'shuffle'])
        else:
            char_warp_select = char_warp
        
        # Перемешиваем ближайшие символы
        if char_warp_select == 'shuffle':
            choice_char = random.randint(0, len(word) - 2)
            word = list(word)
            word[choice_char], word[choice_char + 1] = word[choice_char + 1], word[choice_char]
            word = "".join(word)

        # Добавляем случайный символ в случайное место
        if char_warp_select == 'add':
            choice_char = random.randint(0, len(word) - 1)
            word = word[:choice_char] + random_char + word[choice_char:]

        # Удаляем случайный символ
        if char_warp_select == 'remove':
            choice_char = random.randint(1, len(word))
            word = word[:choice_char - 1] + word[choice_char:]

        # Меняем случайный символ на другой случайный символ
        if char_warp_select == 'replace':
            choice_char = random.randint(0, len(word) - 1)
            word = list(word)
            word[choice_char] = random_char
            word = "".join(word)

        # Уменьшаем значение переменной чтобы выйти из цикла
        warps -= 1
    return(word)

# Проверим работу функции
warp('Москва', warps = 2)

'uskva'

Мы проверили работу функции на слове "Москва", для наглаядности используя две опечатки. На практике мы остановились на том, что для аугментации данных используем только одну опечатку - слишком сильно искажать данные тоже не стоит, иначе модель может работать неадекватно.

В данном случае, у нас получилось слово "uskva" - функция удалила первую букву и заменила вторую. У вас может получиться что-то другое. Теоретиченски, функция может не всегда отрабатывать: она может, скажем, заменить символ на точно такой же или заменить два соседних символа, которые в исконном слове одинаковы, или при выборе параметра warps > 1, функция может заменить дважды один и тот же символ и так далее. 

Это можно было бы предусмотреть, но мы посчитали это нецелесообразным, так как скорее всего функция создаст опечатку и мы получим достаточное количество новых данных.

#### Создаём обучающую выборку

Теперь, собственно, создаём саму обучающую выборку

In [162]:
# Убираем из датафрейма записи, содержащие ссылки на ресурсы в интернете
df_alter = df_alter[~df_alter[ALTERNATE_NAME_COLUMN].str.contains("https://")]

# Объединяем датафреймы df_alter и df_cities_emb по geoname_id так, чтобы в одной колонке были значения из df_alter, а вдругой из 
#df_cities_emb с соответствующем geoname_id

df_learn = df_alter.merge(df_cities_emb, on = GEONAME_ID_COLUMN).copy()

# Проверяем что всё впорядке
print(len(df_learn)), print(len(df_alter))
df_learn.head(5)



28030
28030


,alternate_name,geoname_id,name
0,Andorra,0,Andorra
1,Андорра,0,Andorra
2,United Arab Emirates,-290557,United Arab Emirates
3,Объединенные Арабские Эмираты,-290557,United Arab Emirates
4,Арабские Эмираты,-290557,United Arab Emirates


Мы видим, что всё хорошо - длина df_alter и df_learn совпадают, как и должно быть в поле alternate_name альтернативные названия, а в поле name - унифицированные. Теперь нам надо применить к обучающей выборке функции, которые унифицируют слова и добавят аугментированные слова.

In [163]:

# Унифицируем слова в колонках с истинным и альтернативным именами приводя их к виду (Москва = moskva). 
df_learn[ALTERNATE_NAME_COLUMN] = df_learn[ALTERNATE_NAME_COLUMN].apply(prepare_word) 
df_learn[NAME_COLUMN] = df_learn[NAME_COLUMN].apply(prepare_word) 

df_alter = df_learn.copy()
# Теперь мы будем применять к столбцу с альтернативными названиями функцию, добавляющую опечатки, 
# а затем будем добавлять получившийся результат в обучающий датасет. Если значение WARPS равно нулю, то опечатки добавлены не будут,
# но программа отработает корректно
for i in tqdm(range(WARPS)):
   
# Нам нужен отдельный датафрейм, чтобы применять к нему опечатки и затем соединять с обучающим, иначе каждая последующая итерация будет
# применять опечатки и к тому что добавлено, удваивая датасет
    df_learn_warped = df_alter.copy()
# Применяем опечатки
    df_learn_warped[ALTERNATE_NAME_COLUMN] = df_learn_warped[ALTERNATE_NAME_COLUMN].apply(lambda x: warp(word = x, warps = 1))
# Присоединяем результат к обучающей выборке
    df_learn = pd.concat([df_learn,df_learn_warped])

# При добавлении опечаток больше одного раза могут образоваться дубликаты, так что выкидываем их
df_learn = df_learn.drop_duplicates(subset = ALTERNATE_NAME_COLUMN)


  0%|          | 0/2 [00:00<?, ?it/s]

In [164]:
# Проверяем результат
df_learn.sort_values(by = GEONAME_ID_COLUMN, ascending = False).head(10)


,alternate_name,geoname_id,name
23722,mehzgore,12041452,mezgore
23722,mezhgoer,12041452,mezgore
23722,mezhgore,12041452,mezgore
23458,fedorovskii,11886891,fedorovskiy
23458,fedorovkii,11886891,fedorovskiy
22903,zerzhinskii,8521440,dzerzhinsky
22904,q1n35189,8521440,dzerzhinsky
22904,q135n189,8521440,dzerzhinsky
22903,dgzerzhinskii,8521440,dzerzhinsky
22904,q135189,8521440,dzerzhinsky


Как видим, опечатки были созданы.

## Обучение модели

Напишем функцию для обучения модели

19048 rows × 2 columns

In [165]:


def fit_model(fit_df = df_learn):
    # Подгружаем модель, которую будем обучать
    model = SentenceTransformer(BASE_MODEL)
    # Формируем списки слов из колонки с истинными и альтернативными именами
    sentences1 = fit_df[NAME_COLUMN].to_list()
    sentences2 = fit_df[ALTERNATE_NAME_COLUMN].to_list()
    # Создаём пустой список для последующего хранения в нём InputExample
    train_examples = []

    for i in tqdm(range(len(sentences1))):
    # Заполняем список InputExample. Это особый тип сущностей, которые понимают подели, по сути две строки, которые являются обучабщим примером
        train_examples.append(InputExample(texts=[sentences1[i], sentences2[i]]))
    
    # Инициализируем DataLoader, который запихнём в модель наши данные. Размер батча равен нашей константе.
    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=BATCH_SIZE)
    # Функция потерь, от которой зависит алгоритм обучения модели. Конкретная функция потерь позволяет обучать модель без 
    # меток класса соответствия
    train_loss = losses.MultipleNegativesRankingLoss(model=model)
    # Собственно, обучаем модель и сохраняем её. Путь указан в константе FINAL_MODEL.
    model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=EPOCHS,  
               output_path=FINAL_MODEL)

# Обучаем модель, если указано, что её нужно обучать.
if FIT_MODEL == 0:
    fit_model()
else:
    print('Модель уже обучена')

# Загружаем итоговую модель в любом случае (если вы не обучали, то загружается сохранённая). Если вы брали модель "из коробки", а потом всё же
# решили дообучить её, не забудьте поменять путь.
model = SentenceTransformer(FINAL_MODEL)

  0%|          | 0/65274 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2040 [00:00<?, ?it/s]

## Векторизация текста

Теперь нам нужно векторизовать тексты из той выборки, где будем искать текст, то есть, создать эмбеддинги.

In [166]:
def make_embeddings(df_emb = df_cities_emb, column_name = NAME_COLUMN):
# Так как процесс долгий, он специально разбит на этапы, чтобы был виден прогресс. Для этого разделим длину датасета на 100 с округлением вверх
# и будем по частям сохранять векторы.
    n = int(-1 * len(df_emb) // 100 * -1)
# Также не забудем применить prepare_word к нашим данным. 
    df_emb[column_name] = df_emb[column_name].apply(prepare_word)
    for i in tqdm(range(n)):
# Копируем наш датасет, чтобы работать с копией
        df2 = df_emb.copy()
# Получаем корпус текстов в текущем срезе данных, который зависит от этапа
        corpus = list(df2.iloc[i*100:(i+1)*100][column_name])
# Кодируем текст с помощью модели
        embeddings = model.encode(corpus)
# Преобразуем результат в датафрейм. Затем по индексу соединяем его с соответсвующей частью исходных данных
        df_return=pd.DataFrame(data=embeddings) 
        df_return = pd.merge(df2.iloc[i*100:(i+1)*100].reset_index(drop = True), df_return, left_index=True, right_index=True)
# В случае первой итерации создаём результирующий датафрейм, если итерация не первая - присоединяем результат к результирующему датафрейму
        if i == 0:
            df_result = df_return
        else:
            df_result= pd.concat([df_result,df_return])
    return df_result



Эмбеддинги довольно тяжёлые данные. Нам нужно сохранить их в базу данных, будем делать это по частям (по 1000 записей).

In [167]:
def save_embeddings(df_emb = df_cities_emb, column_name = NAME_COLUMN):
# Вызываем предыдущую функцию, чтобы получить эмбеддинги, которые будем сохранять.
    df_emb = make_embeddings(df_emb, column_name)
# Делим данные на 1000 с округлением вверх
    n = int(-1 * len(df_emb) // 1000 * -1)

    for i in tqdm(range(n)):
# Берём срез данных
        df2 = df_emb.iloc[i*1000:(i+1)*1000]
# Если это первая итерация, то перезаписываем таблицу, если нет, то добавляем к уже существующей
        if i == 0:
            df2.to_sql(EMBEDDINGS, engine, if_exists='replace', index=False)
        else: 
            df2.to_sql(EMBEDDINGS, engine, if_exists='append', index=False)



In [168]:
# Применяем функцию если мы указали соотвествующие опции
if VECTORIZATION == 0:
    save_embeddings(df_emb = df_cities_emb)
else:
    print('Эмбеддинги уже созданы')

# Если мы хотим автоматически обновлять опции, то делаем это. Мы просто присвоим значения 1 в оба столбца, так как при любом раскладе
# это и должно случиться. 
if AUTO_UPDATE_OPTIONS:
    update_database = pd.DataFrame( columns = ['fit_model', 'vectorization'])
    update_database.loc[0] = [1, 1]
    update_database.to_sql(OPTIONS, engine, if_exists='replace', index=False)
    print('Таблица с опциями обновлена')

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Таблица с опциями обновлена


Эмбеддинги созданы. Теперь нужно использовать их для предсказания.

## Предсказание. Тестирование модели

Для начала загрузим наши эмбеддинги.

In [169]:
def read_embeddings():
# Считываем эмбеддинги из базы данных.
    query = f'SELECT *  FROM "{EMBEDDINGS}"'
    df_embeddings= pd.read_sql_query(query, con=engine)

# Преобразуем их в np.Array чтобы модель могла с ними работать. Для начала возьмём из датафрейма только те столбцы, 
# где записаны получившиеся числа, затем преобразуем их в np.array типа float 
    embeddings = df_embeddings.iloc[:, 2:].values.tolist()
    embeddings = np.array(embeddings,dtype='float32')
    return df_embeddings, embeddings

df_embeddings, embeddings = read_embeddings()

### Функция предсказания

По условию задачи, мы хотим возвращать список словарей, в которых будут следующие ключи с соответствующими значениями: <code>geonameid, name, region, country, cosine_similarity</code>. 

In [170]:
def search_df():
    # Запрашиваем заново нужные колонки из таблицы с городами
    query = f'SELECT "{GEONAME_ID_COLUMN}","{NAME_COLUMN}","{ADMIN_CODES_COLUMN}", "{COUNTRY_CODE_COLUMN}"  FROM {CITIES}'
    result_cities = pd.read_sql_query(query, con=engine)
    
    #Для получения стран с заполненными geoname_id запрашиваем функцию, которую мы уже писали, когда генерировали заполненные пропуски
    result_countries = make_countries(save_country_code=True)
    
    # Переименуем колонку iso в country_code
    result_countries = result_countries.rename(columns = {ISO_COLUMN:COUNTRY_CODE_COLUMN})

    # Нам надо получить страну вместо кода страны. Смержим нашу таблицу с городами с таблицей со странами по колоке с кодами страны
    # Так же, уберём geoname_id из второго датасета, так как нас интересует только то, чтобы появилась релевантная колонка со странами
    result = result_cities.merge(result_countries.drop(columns=[GEONAME_ID_COLUMN]), on = COUNTRY_CODE_COLUMN)

    # Добавим в датафрейм со странами колонку name, которая соответсвует стране
    result_countries[NAME_COLUMN] = result_countries[COUNTRY_COLUMN]

    # Объединим наш датафрейм со странами методом конкат, чтобы добавить сами страны
    result = pd.concat([result_countries, result]).fillna('NONE')
    
    # Теперь нам нужно получить регион. В таблице с регионами его код представлен в формате AD.05. Т.е. нам придётся соединить строку
    # с кодом страны и admin1_code, а затем смержить наш датасет с датасетом с регионами по этому столбцу
    result[ADMIN_CODES_COLUMN] = result[COUNTRY_CODE_COLUMN]  +'.'+ result[ADMIN_CODES_COLUMN]
    # Переименуем получившийся столбец
    result = result.rename(columns = {ADMIN_CODES_COLUMN:CODE_COLUMN})
    
    # Подгрузим данные с регионами
    query = f'SELECT "{CODE_COLUMN}", "{REGION_NAME_ASCII_COLUMN}" FROM "{ADMIN_CODES}"'
    result_admin = pd.read_sql_query(query, con=engine)

    # Смержим с нашим датасетом
    result = result.merge(result_admin, on = CODE_COLUMN, how = 'outer')
    
    # В датасете есть куча регионов которые не встречаются у нас, потому появились пропуски в интересующих нас колонках. Дропнем их
    result.dropna(subset=GEONAME_ID_COLUMN, inplace=True)

    # Дропнем колонки code и country_code, т.к. они не нужна
    result.drop(columns = [CODE_COLUMN], inplace=True)
    result.drop(columns = [COUNTRY_CODE_COLUMN], inplace=True)

    # Переименуем name_ascii. В новом имени не нужна константа, так этот датасет генерируется нами
    result = result.rename(columns = {REGION_NAME_ASCII_COLUMN: 'region'})
    result.fillna('NOT', inplace=True)

    return result.sort_values(by=GEONAME_ID_COLUMN)

search_df().head(3)


,country,geoname_id,name,region
259,Netherlands Antilles,-8505032.0,Netherlands Antilles,NOT
203,South Sudan,-7909807.0,South Sudan,NOT
29,"Bonaire, Saint Eustatius and Saba",-7626844.0,"Bonaire, Saint Eustatius and Saba",NOT


Теперь мы получили датафрейм, в котором есть все нужные данные, кроме косинусного расстояния. Его мы получим только в функции предсказания.

In [171]:
search_cities_df = search_df()

def predict(
        # Слово, которое мы будем искать
        word, 
        # Количество сходств, которое мы будем искать
        top_k = 5,
        # Датафрейм из которого будут извлекаться словари
        search_cities_df = search_cities_df,
        # Считанный из базы данных датафрейм с эмбеддингами
        df_embeddings = df_embeddings,
        # Вектор с эмбеддингами
        embeddings = embeddings,
        # Будем ли мы возвращать предсказания в виде словаря, как указано в техническом задании или в виде датафрейма?
        # (Естественно, по умолчанию в виде словаря)
        to_dict = True
        
        ):
    # Обработаем входящее слово
    word = prepare_word(word)
    # С помощью модели рассчитаем ветор для нашего входящего слова
    vector = model.encode([word])
    # Получаем результат нашего предсказания в виде индексов и сходств 
    result = semantic_search(vector,embeddings,top_k = top_k)
    # Инициализируем списки для id наших корпусов и косинусных расстояний
    cos_sims = []
    indexes = []
    # Запишем их в списки
    for i in result[0][:]:
        indexes.append(i['corpus_id'])
        cos_sims.append(i['score'])
    
    # Ищем geoname_id в датафрейме с эмбеддингами, который мы загрузили из базы данных
    pred_df = df_embeddings.iloc[indexes].copy()[GEONAME_ID_COLUMN].to_frame()
    # Добавляем косинусное сходство
    pred_df['cosine_similarity'] = cos_sims
    # Мержим с датафреймом, который мы специально готовили выше
    pred_df = pred_df.merge(search_cities_df, on = GEONAME_ID_COLUMN)
    
    if to_dict:
        return pred_df.to_dict('records') 
    else:
        return pred_df

predict('Влодивасток', top_k = 2)


[{'geoname_id': 2013348,
  'cosine_similarity': 0.7519763112068176,
  'country': 'Russia',
  'name': 'Vladivostok',
  'region': 'Primorye'},
 {'geoname_id': 472459,
  'cosine_similarity': 0.6214302778244019,
  'country': 'Russia',
  'name': 'Vologda',
  'region': 'Vologda Oblast'}]

Наша функция работает как было указано

### Тестирование

Теперь протестируем нашу модель. Заказчик предоставил нам тестовый датасет. Посмотрим на него.

In [172]:
test_df = pd.read_csv(TEST_CSV, sep = ';')
test_df.head()

,query,name,region,country
0,Смоленск,Smolensk,Smolensk Oblast,Russia
1,Кемерово,Kemerovo,Kuzbass,Russia
2,Бишкек,Bishkek,Bishkek,Kyrgyzstan
3,Москва,Moscow,Moscow,Russia
4,Алматы,Almaty,Almaty,Kazakhstan


Нас интересуют поля query, которые содержат запрос и поле name, которое содержит истинное название города. Мы будем делать предсказания по всей тестовой выборке и сравнивать результаты с истинным именем, указанным в текстовом датасете. В качестве результата мы будем возвращать две метрики:
- <b>accuracy</b> - количество совпадений названий из первого места списка.
- <b>accuracy*k</b> - количество попаданий правильного названия в любое место списка длиной k. В данном случае для k возьмём значение 5. 

In [173]:
def test(
        # Название тестового csv (в константе)
        test_csv = TEST_CSV, 
        # Разделитель (там точка с запятой)
        sep = ';',  
        # Длина списка предсказаний
        k = 5):
    
    # Считываем тестовый csv
    test_df = pd.read_csv(test_csv, sep = sep)
    # Конвертируем запросы в список
    querys = test_df['query'].to_list()
    # Наши метрики, о которых было написано выше. Пока они равны нулю
    accuracy = 0
    accuracy_k = 0
    for i in tqdm(range(len(querys))):
        # Получаем предсказание. to_dict ставим False, так как тут удобней работать с датафреймом
        predicted_name = predict(querys[i], top_k = k, to_dict = False).reset_index(drop = True)
        # Сохраняем полученный датафрейм значение в новую переменную
        predicted_name_k = predicted_name
        # В predicted_name сохраняем только предсказанное имя на первой строке
        predicted_name = predicted_name.loc[0][NAME_COLUMN]
        # Забираем истинное название города и преобразуем его в унифицированную форму
        true_name = prepare_word(test_df.loc[i]['name'])
        
        # Сравниваем его с первым предсказанным именем. Если они равны, то даём очко accuracy
        if true_name == prepare_word(predicted_name):
            accuracy += 1
        
        # Ищем истинное имя в списке предсказанных имён на любой позиции (список предсказанных имён ограничен).
        # Если находим то даём очко accuracy_k
        # Заметим, accuracy_k не может быть меньше accuracy, т.к. если верное предсказание на первом месте, то accuracy_k тоже засчитается
        if true_name in predicted_name_k[NAME_COLUMN].apply(prepare_word).to_list():
            accuracy_k += 1

    # Делим полученные значения на длину выборки.
    accuracy = accuracy / len(test_df)
    accuracy_k = accuracy_k / len(test_df)
    return accuracy, accuracy_k

accuracy, accuracy_k = test(test_csv = TEST_CSV, sep = ';', k = TEST_K)


  0%|          | 0/345 [00:00<?, ?it/s]

In [174]:
print(f'Значение accuracy = {accuracy:,.3f}, значение accuracy*{TEST_K} = {accuracy_k:,.3f}')


Значение accuracy = 0.925, значение accuracy*5 = 0.939


Мы протестировали нашу модель, она показала значения accuracy = 0.925, значение accuracy*5 = 0.939 на тестовой выборке. 

## Вывод

В ходе исследования были испробованы такие подходы, как TfidfVectorizer и написание собственной нейронной сети, но в конце концов было решено остановиться на предобученной нейронной сети Sentence Transformers. Конкретно за основу нашей модели была взята distiluse-base-multilingual-cased-v2 и дообучена на альтернативных названиях городов, при том сам поиск производится только на унифицированным названиям городов. Был испробован альтернативный вариант - искать не только по унифицированным названиям городов, но и по альтернативным, ведь мы знаем их geoname_id и можем сопоставить с унифицированными названиями. Однако, такой подход имеет существенный недостаток - поиск происходит довольно медленно, что неприемлемо для целей заказчика.

Так же, исследования показали, что предварительная обработка слов и добавление в обучающую выборку аугментированных данных очень существенно улучшают результаты работы модели. 

#### Были выполнены следующие задачи

- Мы настроили подключение к базе данных. Таким образом, программа работает непосредственно с базой данных заказчика.
- Были созданы настройки, благодаря которым заказчик сможет использовать любые данные схожей структуры. Он сможет использовать свои названия для схем данных и колонок, таким образом ему не придётся вмешиваться в код программы, чтобы подстроить её под свои данные или же менять названия своих данных (но важно чтобы структура данных сохранялась)
- Программа записывает в базу данных заказчика готовые эмбеддинги, таким образом, ему не придётся каждый раз заново рассчитывать их.
- Был создан скрипт, который создаёт и считывает дополнительную схему данных, в которой указывается, требуется ли повторное обучение модели и создание эмбеддингов. Более того, программа может считать эти настройки из базы данных, таким образом не обязательно каждый раз менять значения соответствующих констант.
- Была самостоятельно создана таблица с дополнительными альтернативными названиями для стран.
- Были созданы настройки обучения модели. Таким образом, заказчик может поменять параметры обучения просто поменяв значения некоторых констант.
- Был создан (также настраиваемый) скрипт для предподготовки данных (который включает в себя и аугментацию)
- Была обучена модель, основанная на  distiluse-base-multilingual-cased-v2. Обучение длилось 4 эпохи. Модель показала результаты accuracy = 0.925, значение accuracy*5 = 0.939. Она умеет справляться с опечатками, как было сказано в ТЗ.
- Была создана финальная функция predict(), которая принимает слово и возвращает список словарей, соответсвующий техническому задания. При том, в функции есть настройка, позволяющая выдавать вместо списка словарей датафрейм, что может быть удобней для тестирования непосредственно в теле программы. Также, согласно ТЗ, в функцию добавлен параметр, определяющий длину возвращаемого списка.
- Модель также умеет определять страны, не только города.




## Что может улучшить модель?

<b>Со стороны заказчика</b>. Согласно заданию, модель должна помогать найти унифицированное название города среди вакансий, но сам поиск проводится вручную, программа лишь помогает тем, что выдаёт список из возможных ближайших городов. Соответственно, можно настроить скрипт, который будет автоматически заносить в некую новую таблицу данных все пользовательские названия городов и соответствующие ему унифицированные названия городов. Затем, раз в какое-то время сверять её с базой с альтернативными городами и добавлять туда все несовпадающие названия, таким образом можно постепенно расширять базу альтернативных городов и соответственно расширять обучающую выборку.

<b>Со стороны разработчика</b>. Качество модели можно улучшать бесконечно, так как нет предела совершенству, но вот несколько нереализованных пока идей:
- Найти более подходящий способ унифицировать слова. Возможно, использовать более продвинутые библиотеки для перевода и транслитерации (такие библиотеки предлагались, но они иногда вызывали ошибки, так как не всегда могли определить язык)
- Добавить в поисковую выборку некоторое количество альтернативных названий, например таких, с которыми модель видит наименьшее сходство. Таким образом, добиться некоторого компромисса - и расширить поисковую выборку, и не слишком потерять в скорости.
- Подробно проанализировать ошибки, понять слабые места модели, возможно, самостоятельно дополнить обучающую выборку.
- Перепробовать больше моделей с разными параметрами.